In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

RTS_DATA = "/mnt/d/rts/"
METADATA = RTS_DATA + 'metadata'
VIDEOS = RTS_DATA + '0'
REMOTE_VIDEOS = "/mnt/rts"
OUTDIR = 'data'



/home/kikohs/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple

In [3]:
# LOCAL imports
import rts

LOG = rts.utils.get_logger()

/home/kikohs/.cache/pypoetry/virtualenvs/rts-DIiCth0j-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# vidx = rts.ingest.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.ingest.read_all_metadata(METADATA, vidx)
# rts.ingest.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.ingest.load_metadata_hdf5(OUTDIR, 'rts_metadata')

In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)
midx.get_stats().number_of_documents

In [ ]:
#TODO get unique geo, people, for the whole media

In [ ]:
a = df.groupby('collection')['collection'].count().sort_values(ascending=False)
a.to_csv(os.path.join(OUTDIR, 'collections.csv'))

In [ ]:
a = df.groupby('contentType')['contentType'].count().sort_values(ascending=False)
a.to_csv(os.path.join(OUTDIR, 'contentType.csv'))

In [ ]:
midx.delete_all_documents()

In [ ]:
rts.db.fulltext.index_media(df, midx)

In [34]:
r = rts.io.archive.transcribe_media(os.path.join(OUTDIR, 'myaudio.m4a'))

In [ ]:
df.publishedBy.apply(lambda x: len(x) if x else 0).unique()

In [ ]:
# midx = rts.db.meilisearch.get_or_create_media_index(client)
midx.get_stats().number_of_documents

In [ ]:
%ls {VIDEOS}

In [47]:
video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)

rts.io.archive.create_optimized_media(video_part)


'/mnt/d/rts/0/0/0/ZB001000/export'

In [38]:
a = os.path.join(VIDEOS, '0', '0', 'ZB002000', 'export', 'ZB002000_audio.m4a')
r = rts.io.archive.transcribe_media(a, 'small')

INFO:RTS:Load model: small


In [40]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

True

In [19]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

INFO:RTS:to_mp3 => 59664.23749923706 ms


'data/audio.mp3'

In [34]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

INFO:RTS:to_mp3 => 59204.33855056763 ms


'data/audio.mp3'

In [38]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

INFO:RTS:remux_audio => 343.6279296875 ms


'data/myaudio.m4a'

In [7]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    print(stream.codec_context.height)
    # for frame in container.decode(stream):

    #     print(frame)
    #     print(frame.time, frame.time_base)
    #     # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
    #     frame.to_image().save(
    #         os.path.join(out_path, f"night-sky.{frame.pts:04d}.jpg"),
    #         quality=80,
    #     )

/mnt/d/rts/0/0/0/ZB001000/ZB001000_track1_dashinit.mp4
576
